<a href="https://colab.research.google.com/github/jovanadobreva/Labs-I2DS/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In [3]:
!pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [7]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: C:\Users\zhark\Downloads\ElectricCarData.csv

  0%|          | 0.00/8.20k [00:00<?, ?B/s]
100%|##########| 8.20k/8.20k [00:00<?, ?B/s]


### Import the required libraries

In [11]:
import pandas as pd

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [104]:
data = pd.read_csv('ElectricCarData.csv')

In [105]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


### Encode string variables

In [106]:
from sklearn.preprocessing import LabelEncoder
columns = ['Brand','Model','RapidCharge','PowerTrain','PlugType','BodyStyle','Segment']
for i in columns:
    le = LabelEncoder()
    data[i] = le.fit_transform(data[i])


In [113]:
data['FastCharge_KmH'].unique()
data.replace('-', pd.NA, inplace=True)
# data['FastCharge_KmH'].unique()

array(['940', '250', '620', '560', '190', '220', '420', '650', '540',
       '440', '230', '380', '210', '590', '780', '170', '260', '930',
       '850', '910', '490', '470', '270', '450', '350', '710', '240',
       '390', '570', '610', '340', '730', '920', <NA>, '550', '900',
       '520', '430', '890', '410', '770', '460', '360', '810', '480',
       '290', '330', '740', '510', '320', '500'], dtype=object)

In [127]:
# data.isnull().sum()
# data['FastCharge_KmH'].isnull().sum()
mean_value = data['FastCharge_KmH'].dropna().astype(float).mean()
data['FastCharge_KmH'] = data['FastCharge_KmH'].replace({pd.NA: None})
# print(mean_value)
data['FastCharge_KmH'] = data['FastCharge_KmH'].astype(float).fillna(mean_value)
# print(data['FastCharge_KmH'].dtypes)
# data['FastCharge_KmH'] = pd.to_numeric(data['FastCharge_KmH'], errors='coerce')
# data.fillna(data.mean(numeric_only=True), inplace=True)
# data['FastCharge_KmH'] = data['FastCharge_KmH'].fillna(data['FastCharge_KmH'].mean())

## Split the dataset for training and testing in ratio 80:20

In [128]:
from sklearn.model_selection import train_test_split
X = data.drop('PriceEuro',axis=1)
y = data['PriceEuro']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.dtypes)
print(y_train.dtypes)

Brand                int32
Model                int32
AccelSec           float64
TopSpeed_KmH         int64
Range_Km             int64
Efficiency_WhKm      int64
FastCharge_KmH     float64
RapidCharge          int32
PowerTrain           int32
PlugType             int32
BodyStyle            int32
Segment              int32
Seats                int64
dtype: object
int64


## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [137]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=5,min_samples_split=10,max_features=sqrt)
model.fit(X_train,y_train)


NameError: name 'sqrt' is not defined

## Predict the outcomes for X test

In [136]:
y_pred = model.predict(X_test)

Mean of y_test: 61268.142857142855


## Assess the model performance, by using sklearn metrics for regression

In [138]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R²):", r2)

Mean Absolute Error (MAE): 9507.657142857144
Mean Squared Error (MSE): 227067329.4780629
Root Mean Squared Error (RMSE): 227067329.4780629
R-squared (R²): 0.7136739593669699


## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [141]:
from xgboost import XGBRegressor
XGB_model = XGBRegressor(n_estimators=100,max_depth=5,learning_rate=0.1,objective="reg:squarederror")
XGB_model.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Predict the outcomes for X test

In [143]:
XGB_pred = XGB_model.predict(X_test)

## Assess the model performance, by using sklearn metrics for regression

In [144]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
XGB_mae = mean_absolute_error(y_test, XGB_pred)
XGB_mse = mean_squared_error(y_test, XGB_pred)
XGB_rmse = mean_squared_error(y_test, XGB_pred)
XGB_r2 = r2_score(y_test, XGB_pred)

print("XGBRegressor performance")
print("Mean Absolute Error (MAE):", XGB_mae)
print("Mean Squared Error (MSE):", XGB_mse)
print("Root Mean Squared Error (RMSE):", XGB_rmse)
print("R-squared (R²):", XGB_r2)


XGBRegressor performance
Mean Absolute Error (MAE): 8233.73400297619
Mean Squared Error (MSE): 243235397.98703295
Root Mean Squared Error (RMSE): 243235397.98703295
R-squared (R²): 0.6932864189147949


## Compare the performances of both model for at least three regression metircs

In [145]:
print("Comparison between 2 models(DecisionTree and XGBRegressor)")
print(f"DecisionTreeRegressor - MAE: {mae}, MSE: {mse}, RMSE: {rmse}, R2: {r2}")
print(f"XGBRegressor - MAE: {XGB_mae}, MSE: {XGB_mse}, RMSE: {XGB_rmse}, R2: {XGB_r2}")

Comparison between 2 models(DecisionTree and XGBRegressor)
DecisionTreeRegressor - MAE: 9507.657142857144, MSE: 227067329.4780629, RMSE: 227067329.4780629, R2: 0.7136739593669699
XGBRegressor - MAE: 8233.73400297619, MSE: 243235397.98703295, RMSE: 243235397.98703295, R2: 0.6932864189147949
